In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
import pickle
import youtokentome as yttm
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import ComplementNB
import numpy as np
from sklearn import metrics

In [3]:
with open('bow-language-detection.pkl', 'rb') as fopen:
    bow = pickle.load(fopen)

In [4]:
with open('train-test.json') as fopen:
    train_test = json.load(fopen)
    
train_test.keys()

dict_keys(['train_X', 'test_X', 'train_Y', 'test_Y'])

In [5]:
test_Y = LabelEncoder().fit_transform(train_test['test_Y'])

In [6]:
bpe = yttm.BPE(model='language-detection.model')

In [7]:
test_subs = [' '.join(s) for s in bpe.encode(train_test['test_X'], output_type=yttm.OutputType.SUBWORD)]
len(test_subs)

2273214

In [8]:
test_X = bow.transform(test_subs)

In [9]:
import tensorflow as tf

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [10]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensorValue(indices, coo.col, coo.shape), tf.SparseTensorValue(indices, coo.data, coo.shape)

In [11]:
class Model:
    def __init__(self, learning_rate, dimension = 32, output = 6):
        self.X = tf.sparse_placeholder(tf.int32)
        self.W = tf.sparse_placeholder(tf.int32)
        self.Y = tf.placeholder(tf.int32, [None])
        embeddings = tf.Variable(tf.truncated_normal([400000,dimension]))
        embed = tf.nn.embedding_lookup_sparse(embeddings, self.X, self.W, combiner='mean')
        self.embed = embed
        self.logits = tf.layers.dense(embed, output)

In [12]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(1e-3)
sess.run(tf.global_variables_initializer())

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'lang-detection-w/model.ckpt')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from lang-detection-w/model.ckpt


In [14]:
from tqdm import tqdm

batch_size = 128
predicted_Y = []

for i in tqdm(range(0, test_X.shape[0], batch_size)):
    index = min(i + batch_size, test_X.shape[0])
    batch_x = convert_sparse_matrix_to_sparse_tensor(test_X[i: index])
    results = sess.run(model.logits, feed_dict = {
                model.X: batch_x[0],
                model.W: batch_x[1],
            })
    predicted_Y.extend(np.argmax(results, axis = 1))

100%|██████████| 17760/17760 [00:31<00:00, 557.61it/s]


In [15]:
len(test_Y) == len(predicted_Y)

True

In [16]:
print(
    metrics.classification_report(
        test_Y,
        predicted_Y,
        target_names = ['eng', 'ind', 'malay', 'manglish', 'other', 'rojak'],
        digits = 5
    )
)

              precision    recall  f1-score   support

         eng    0.97730   0.98195   0.97962    110129
         ind    0.97384   0.95950   0.96662    354327
       malay    0.97534   0.98862   0.98194   1405848
    manglish    0.98968   0.98971   0.98969    182420
       other    0.98128   0.97775   0.97951     78259
       rojak    0.95718   0.86179   0.90698    142231

    accuracy                        0.97554   2273214
   macro avg    0.97577   0.95989   0.96739   2273214
weighted avg    0.97542   0.97554   0.97529   2273214

